In [18]:
import numpy as np
import os 
import pandas as pd

import sqlite3

In [19]:
# display full data of dataframe using set_option method
# pd.set_option("display.max_rows", None, "display.max_columns", None)
# pd.reset_option('all')

### Naive version of storing CREATE & INSERT statements of each db_id

#### .sql

In [20]:
# scan the directory and store the filenames
files = []

for dirname, _, filenames in os.walk('/Users/kunwooshin/Desktop/VLDB/nl2sql/spider/database'):
    for filename in filenames:
        files.append(dirname+'/'+filename)

ddl_files = [filename for filename in files if filename.endswith(".sql")]

# extract the db_id from the filename
db_id = []

for i in range(len(ddl_files)):
    file_path = ddl_files[i]
    db_id.append(file_path.split('/')[8])
    
# initilalize ddl table    
ddl_tbl = pd.DataFrame({'db_id':db_id, 'CREATE':'', 'INSERT':''})

# append CREATE TABLE and INSERT queries for each db_id
for i in range(len(ddl_files)):
    create_script = ''
    insert_script = ''

    with open(ddl_files[i], 'r') as f:
        for line in f:
            if not any(keyword in line for keyword in ['PRAGMA', 'BEGIN TRANSACTION', 'COMMIT', '--', 'DROP', '***', '<']):
                if ('INSERT' in line) or ('insert' in line):
                    insert_script += line
                else:
                    create_script += line
                    
    ddl_tbl.iloc[i, 1], ddl_tbl.iloc[i, 2] = create_script.replace('\n',' ').replace('\t', ' '), insert_script.replace('\n',' ').replace('\t', ' ')

In [21]:
# db_id which has INSERT statements on the seperate .txt file
dir_name = '/Users/kunwooshin/Desktop/VLDB/nl2sql/spider/database/'
db_id_manual = ['car_1', 'flight_2', 'wine_1', 'student_1']
file_path_manual = [dir_name + db_id + '/' + db_id + '.txt' for db_id in db_id_manual]

for i in range(len(file_path_manual)):
    insert_script = ''
    with open(file_path_manual[i], 'r') as f:
        for line in f:
            if ('INSERT' in line):
                insert_script += line
                ddl_tbl.loc[(ddl_tbl['db_id'] == db_id_manual[i]), 'INSERT']= insert_script

In [22]:
ddl_tbl

,db_id,CREATE,INSERT
0,browser_web,"CREATE TABLE IF NOT EXISTS ""Web_client_acceler...","INSERT INTO Web_client_accelerator VALUES(1,'C..."
1,musical,"CREATE TABLE ""musical"" ( ""Musical_ID"" int, ""N...","INSERT INTO ""musical"" VALUES (1,""The Phantom ..."
2,farm,"CREATE TABLE ""city"" ( ""City_ID"" int, ""Offici...","INSERT INTO ""city"" VALUES (1,""Grand Falls/Gra..."
3,game_injury,"CREATE TABLE ""stadium"" ( ""id"" int, ""name"" te...","INSERT INTO ""stadium"" VALUES (1,""Cowboys Stad..."
4,hospital_1,"CREATE TABLE list ( LastName TEXT, FirstNa...","INSERT INTO list (LastName, FirstName, Grade, ..."
...,...,...,...
153,ship_mission,"CREATE TABLE ""mission"" ( ""Mission_ID"" int, ""S...","INSERT INTO ""ship"" VALUES (1,""Corbridge"",""Car..."
154,student_assessment,CREATE TABLE Addresses ( address_id INTEGER N...,"INSERT INTO Addresses (`address_id`, `line_1`,..."
155,assets_maintenance,CREATE TABLE `Third_Party_Companies` ( `compa...,INSERT INTO Third_Party_Companies (`company_id...
156,body_builder,"CREATE TABLE ""body_builder"" ( ""Body_Builder_I...","INSERT INTO ""people"" VALUES (1,""Jack Campbell..."


#### .sqlite

In [23]:
dir_name = '/Users/kunwooshin/Desktop/VLDB/nl2sql/spider/database/'
db_id_lite = ['chinook_1', 'epinions_1', 'icfp_1', 'small_bank_1', 'twitter_1', 'voter_1', 'formula_1', 'csu_1']
file_path_lite = [dir_name + db_id  for db_id in db_id_lite]

files_lite = []
for file_path in file_path_lite:
    files_lite += (os.listdir(file_path))

ddl_files_lite = [filename for filename in files_lite if filename.endswith(".sqlite")]

ddl_tbl_lite = pd.DataFrame({'db_id':db_id_lite, 'CREATE':'', 'INSERT':None})

for i in range(len(ddl_files_lite)):
    create_script = ''
    insert_script_lite = ''
    
    conn = sqlite3.connect(dir_name + db_id_lite[i] + '/' + ddl_files_lite[i])
    cur = conn.cursor()
    
    # extract CREATE TABLE queries
    cur.execute("SELECT sql FROM sqlite_master WHERE type='table'")
    while True:
        line = cur.fetchone()
        if line is None:
            break
        create_script += (line[0] + '; ')
    
    # extract sample rows of tables for each db_ib
    cur.execute("SELECT name FROM sqlite_master WHERE type='table'")
    table_list = cur.fetchall()

    for (table_name,) in table_list:
        rows = []
        
        query = f"SELECT * FROM {table_name}" 
        cur.execute(query)
        
        while True:
            line = cur.fetchone()
            if line is None:
                break
            insert_q = f"INSERT INTO {table_name} VALUES {line}; \n"
            insert_script_lite += insert_q

    conn.close()
    
    ddl_tbl_lite.iloc[i, 1], ddl_tbl_lite.iloc[i, 2] = create_script.replace('\n',' '), insert_script_lite.replace('\n',' ')

In [24]:
ddl_tbl_lite

,db_id,CREATE,INSERT
0,chinook_1,CREATE TABLE `Album` ( `AlbumId` integer NOT...,"INSERT INTO Album VALUES (1, 'For Those About ..."
1,epinions_1,CREATE TABLE `item` ( `i_id` integer NOT NUL...,"INSERT INTO item VALUES (0, 'pear'); INSERT I..."
2,icfp_1,"CREATE TABLE Inst ( instID INTEGER, name T...","INSERT INTO Inst VALUES (1000, 'University of ..."
3,small_bank_1,CREATE TABLE ACCOUNTS ( custid BIGINT...,"INSERT INTO ACCOUNTS VALUES (1, 'Brown'); INS..."
4,twitter_1,CREATE TABLE `follows` ( `f1` int(11) NOT NU...,"INSERT INTO follows VALUES (1, 2); INSERT INT..."
5,voter_1,CREATE TABLE `AREA_CODE_STATE` ( `area_code`...,"INSERT INTO AREA_CODE_STATE VALUES (201, 'NJ')..."
6,formula_1,"CREATE TABLE ""circuits"" ( ""circuitId"" INTEGER ...","INSERT INTO circuits VALUES (1, 'albert_park',..."
7,csu_1,"CREATE TABLE ""Campuses"" ( \t""Id"" INTEGER PRIMA...","INSERT INTO Campuses VALUES (1, 'California St..."


In [25]:
ddl_tbl_concat = pd.concat([ddl_tbl, ddl_tbl_lite], ignore_index=True)

In [26]:
ddl_tbl_concat.shape

(166, 3)

In [37]:
ddl_tbl_concat.to_csv('ddl_tbl.csv')